# Hyperparameter Tuning YOLO

YOLOv8 is the newest YOLO model that can be used for object detection, image classification and segmentation. YOLO stands for __You Only Look Once__ and is a series of models that are very popular and widely used within computer vision. YOLOv8 is known to often outperform it's competitors when it comes to prediction accuracy and speed.

Hyperparameters are high-level parameters that influence the learning process during model training. It is an important step when building a machine learning model to find the optimal set of hyperparameters in order to produce the best possible predictions from a model. The process of finding these optimal values is called hyperparameter tuning. 

In this blog, we combine the popular object detection model YOLOv8 with AWS SageMaker's capabilities to find the optimal hyperparameter configuration.

In [ ]:
# importing the libraries

import sagemaker

from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import CategoricalParameter, ContinuousParameter
from sagemaker.tuner import HyperparameterTuner, HyperbandStrategyConfig, StrategyConfig
from sagemaker.estimator import Estimator

sagemaker_session = sagemaker.Session()
role = get_execution_role()

SageMaker Estimators give a high-level interface for SageMaker training. Details can be found [here](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [ ]:
# setting the metric definitions

metric_definitions=[
    {
        "Name": "precision",
        "Regex": "YOLO Metric metrics/precision\\(B\\): (.*)"
    },
    {
        "Name": "recall",
        "Regex": "YOLO Metric metrics/recall\\(B\\): (.*)"
    },
    {
        "Name": "mAP50",
        "Regex": "YOLO Metric metrics/mAP50\\(B\\): (.*)"
    },
    {
        "Name": "mAP50-95",
        "Regex": "YOLO Metric metrics/mAP50-95\\(B\\): (.*)"
    },
    {
        "Name": "box_loss",
        "Regex": "YOLO Metric val/box_loss: (.*)"
    },
    {
        "Name": "cls_loss",
        "Regex": "YOLO Metric val/cls_loss: (.*)"
    },
    {
        "Name": "dfl_loss",
        "Regex": "YOLO Metric val/dfl_loss: (.*)"
    }
]

We set up an estimator, this is where we give the model an entry point (the training module where the model is stored),

For this example

 set the instance type, a max run time (timeout in seconds for training), after this time Amazon SageMaker terminates the job regardless of its current status. Any hyperparameters you want to have the same value for the whole of training jobs initialise the estimator with can be added here.

The source dir needs to be the directory of which the train.py file is saved.

In [ ]:
estimator = PyTorch(
    entry_point="train.py",
    role=role,
    image_uri='your/image',  # your image
    source_dir="./src",
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    framework_version="1.12.1",
    py_version="py38",
    sagemaker_session=sagemaker_session,
    hyperparameters={},
    use_spot_instances=True,
    input_mode='File',  # FastFile causes a issue with writing label cache
    debugger_hook_config=False,
    max_wait=360000+3600,
    max_run=360000,
    output_path='path/to/output',
    enable_sagemaker_metrics=True,
    metric_definitions=metric_definitions,
)

Once the estimator has been initialised, we can specify the hyperparameters you would like to tune and the possible ranges for their values. As an example, this blog shows a couple of hyperparameters you may want to tune when working with an object detection algorithm

Categorical parameters, ContinuousParameters and IntegerParameters

In [ ]:
hyperparameter_ranges={
    'epochs':IntegerParameter(100, 300),
    'optimizer':CategoricalParameter(['SGD', 'Adam', 'AdamW', 'RMSProp']),
    'lr0': ContinuousParameter(0.00001, 0.01),
    'lrf': ContinuousParameter(0.00001, 0.01),
    'momentum': ContinuousParameter(0.9, 0.9999),
    'weight_decay': ContinuousParameter(0.0003, 0.00099)
}

Want to create a HyperparameterTuner which takes the following inputs:
- the objecttive metric and definition 
    - here we have chosen to maximise the mean average precision mAP
    - hyperparamete ranges
    - the strategy


In [ ]:
tuner = HyperparameterTuner(estimator, 
                            objective_metric_name="mAP50-95", 
                            metric_definitions=metric_definitions, 
                            hyperparameter_ranges= hyperparameter_ranges, 
                            strategy='Hyperband',
                            max_jobs=50,
                            strategy_config = StrategyConfig(hyperband_strategy_config =HyperbandStrategyConfig(max_resource=10, min_resource = 1))
                           )

Want to fit the tuner by passing in the S3 paths to the training dat

In [ ]:
tuner.fit('S3/path/to/training-data')

This starts up a hyperparameter tuning job